In [1]:
import pandas as pd
import numpy as np

In [2]:
car_path = 'C:\\Users\\beam_\\OneDrive\\Desktop\\Masters\\DemographicDataset\\raw\\car availability.csv'
ethnicity_path = 'C:\\Users\\beam_\\OneDrive\\Desktop\\Masters\\DemographicDataset\\raw\\ethnicity.csv'
dependency_path = 'C:\\Users\\beam_\\OneDrive\\Desktop\\Masters\\DemographicDataset\\raw\\household deprivation.csv'
room_occupancy_path = 'C:\\Users\\beam_\\OneDrive\\Desktop\\Masters\\DemographicDataset\\raw\\room_occupancy.csv'

car_df = pd.read_csv(car_path)
ethnicity_df = pd.read_csv(ethnicity_path)
dependency_df = pd.read_csv(dependency_path)
room_occupancy_df = pd.read_csv(room_occupancy_path)

In [3]:
car_df.rename(columns=lambda x: x.strip(), inplace=True)
ethnicity_df.rename(columns=lambda x: x.strip(), inplace=True)
dependency_df.rename(columns=lambda x: x.strip(), inplace=True)
room_occupancy_df.rename(columns=lambda x: x.strip(), inplace=True)

# Standardize column name for joining
car_df.rename(columns={"Output Areas Code": "Output_Areas_Code"}, inplace=True)
ethnicity_df.rename(columns={"Output Areas Code": "Output_Areas_Code"}, inplace=True)
dependency_df.rename(columns={"Output Areas Code": "Output_Areas_Code"}, inplace=True)
room_occupancy_df.rename(columns={"Output Areas Code": "Output_Areas_Code"}, inplace=True)

# Drop repeating column
dependency_df = dependency_df.drop("Output Areas", axis=1)
car_df = car_df.drop("Output Areas", axis=1)

In [4]:
room_occupancy_df.head()

,Output_Areas_Code,Total: All households,Occupancy rating of rooms: +2 or more,Occupancy rating of rooms: +1,Occupancy rating of rooms: 0,Occupancy rating of rooms: -1,Occupancy rating of rooms: -2 or less
0,E00000138,128,26.0,27.0,47.0,21.0,7.0
1,E00000156,134,28.0,34.0,35.0,26.0,11.0
2,E00000157,120,23.0,30.0,36.0,19.0,12.0
3,E00000158,155,27.0,44.0,46.0,31.0,7.0
4,E00000160,130,29.0,30.0,39.0,26.0,6.0


In [6]:
# Pivot dependency DataFrame
pivoted_dependency_df = dependency_df.pivot(
    index="Output_Areas_Code",
    columns="Household deprivation (6 categories)",
    values="Observation"
)
pivoted_dependency_df.columns = [
    f"Deprivation_Category_{col}" if col != -8 else "Does_Not_Apply"
    for col in pivoted_dependency_df.columns
]
pivoted_dependency_df.reset_index(inplace=True)

# Pivot car DataFrame
pivoted_car_df = car_df.pivot(
    index="Output_Areas_Code",
    columns="Car or van availability (4 categories)",
    values="Observation"
)
pivoted_car_df.columns = [
    f"Car_Availability_{col}" for col in pivoted_car_df.columns
]
pivoted_car_df.reset_index(inplace=True)

# Merge all DataFrames
merged_df = (
    pivoted_car_df
    .merge(ethnicity_df, on="Output_Areas_Code", how="outer")
    .merge(pivoted_dependency_df, on="Output_Areas_Code", how="outer")
    .merge(room_occupancy_df, on="Output_Areas_Code", how="outer")
)
merged_df.head()

,Output_Areas_Code,Car_Availability_1 car or van in household,Car_Availability_2 or more cars or vans in household,Car_Availability_Does not apply,Car_Availability_No cars or vans in household,Total: All usual residents,"Asian, Asian British or Asian Welsh: Bangladeshi","Asian, Asian British or Asian Welsh: Chinese","Asian, Asian British or Asian Welsh: Indian","Asian, Asian British or Asian Welsh: Pakistani",...,Deprivation_Category_Household is deprived in one dimension,Deprivation_Category_Household is deprived in three dimensions,Deprivation_Category_Household is deprived in two dimensions,Deprivation_Category_Household is not deprived in any dimension,Total: All households,Occupancy rating of rooms: +2 or more,Occupancy rating of rooms: +1,Occupancy rating of rooms: 0,Occupancy rating of rooms: -1,Occupancy rating of rooms: -2 or less
0,E00000001,41.0,7.0,0.0,46.0,176.0,0.0,4.0,0.0,1.0,...,22.0,0.0,3.0,66.0,94,78.0,13.0,2.0,1.0,0.0
1,E00000003,47.0,11.0,0.0,51.0,255.0,0.0,6.0,11.0,0.0,...,30.0,0.0,5.0,74.0,109,89.0,11.0,7.0,2.0,0.0
2,E00000005,18.0,6.0,0.0,37.0,112.0,0.0,4.0,4.0,0.0,...,15.0,0.0,2.0,48.0,61,13.0,33.0,13.0,2.0,0.0
3,E00000007,16.0,4.0,0.0,66.0,144.0,0.0,16.0,1.0,0.0,...,15.0,1.0,1.0,69.0,86,7.0,34.0,41.0,4.0,0.0
4,E00000010,17.0,0.0,0.0,109.0,175.0,1.0,3.0,3.0,0.0,...,56.0,7.0,22.0,40.0,124,2.0,4.0,91.0,24.0,3.0


In [7]:
# Save the cleaned and merged DataFrame
output_path = 'C:\\Users\\beam_\\OneDrive\\Desktop\\Masters\\DemographicDataset\\merged_data.csv'
merged_df.to_csv(output_path, index=False)